# Tsunami Potential Prediction with Artificial Neural Networks

Model and algorithm by Astri Novianty.

The final proposed backpropagation ANN architecture with 2 hidden-neurons:

![ANN](ann-architecture.png)

In [1]:
import torch

Raw inputs:

* `in_bias: float` = 1
* `t0: float`: Unnormalized rupture duration variable ($T_0$)
* `td: float`: Unnormalized P-wave dominant period variable ($T_d$)
* `t0xtd: float`: Unnormalized product of rupture duration and P-wave dominant period ($T_0 \times T_d$)
* `mw: float`: Unnormalized moment magnitude ($M_w$)

In [2]:
# Case study: M 7.8 - Southwest of Sumatra, Indonesia - 2016-03-02T12:49:48.000Z
# Label
in_name = 'M 7.8 - Southwest of Sumatra, Indonesia - 2016-03-02T12:49:48.000Z'
# Raw inputs
in_bias: float = 1
# Unnormalized rupture duration variable
t0: float = 76.21047386
# Unnormalized P-wave dominant period variable
td: float = 3.484952633
# Unnormalized product of rupture duration and P-wave dominant period (T_0 \times T_d)
t0xtd: float = t0 * td
# Unnormalized moment magnitude (M_w)
mw: float = 7.4894408801871

In [51]:
# Normalization vectors
ri_min = torch.tensor([0, 9.45, 2.7, 42.54644, 6.987921])
ri_max = torch.tensor([1, 203.8708, 7.5, 877.4648, 8.995652])
ri_min, ri_max

(tensor([ 0.0000,  9.4500,  2.7000, 42.5464,  6.9879]),
 tensor([  1.0000, 203.8708,   7.5000, 877.4648,   8.9957]))

In [52]:
t0_norm = (t0 - ri_min[1]) / (ri_max[1] - ri_min[1])
td_norm = (td - ri_min[2]) / (ri_max[2] - ri_min[2])
t0xtd_norm = (t0xtd - ri_min[3]) / (ri_max[3] - ri_min[3])
mw_norm = (mw - ri_min[4]) / (ri_max[4] - ri_min[4])
x = torch.tensor([in_bias, t0_norm, td_norm, t0xtd_norm, mw_norm])
x

tensor([1.0000, 0.3434, 0.1635, 0.2671, 0.2498])

In [53]:
import torch.nn as nn
import torch.nn.functional as F

n_in = 5
n_hidden = 2
n_out = 2

# class NoviantyModel(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.hidden1 = nn.Linear(n_hidden)
#         self.out = nn.Linear(n_hidden, n_out)
        
#     def forward(self, x):
#         F.sigmoid(self.hidden1(x))

# def init_weights(m: nn.Module):
#     if type(m) == nn.Linear:
#         print(m.weight)
#     print(m)

in_layer = torch.nn.Linear(n_in, n_hidden, bias=False)
hidden1_layer = torch.nn.Linear(n_hidden, n_out, bias=False)
model = torch.nn.Sequential(in_layer,
                            torch.nn.Sigmoid(),
                            hidden1_layer,
                            torch.nn.Sigmoid())

in_layer_pretrained = {
    'weight': torch.tensor([
        [1.0, 0.0, 0.0, 0.0, 0.0],
        [132.70666017759126, -70.98121913774737, -130.83160560195375, -
20.91881955841453, -101.27274912451277]
    ])}
hidden1_layer_pretrained = {
    'weight': torch.tensor([
        [1.862494811604578, -1.862494811604578],
        [-4.199132605391263, 4.199132605391263]
    ])}
in_layer.load_state_dict(in_layer_pretrained)
hidden1_layer.load_state_dict(hidden1_layer_pretrained)
print('Input state: ', in_layer.state_dict())
print('Hidden1 state: ', hidden1_layer.state_dict())
#init_weights(list(model.children())[0])

Input state:  OrderedDict([('weight', tensor([[   1.0000,    0.0000,    0.0000,    0.0000,    0.0000],
        [ 132.7067,  -70.9812, -130.8316,  -20.9188, -101.2728]]))])
Hidden1 state:  OrderedDict([('weight', tensor([[ 1.8625, -1.8625],
        [-4.1991,  4.1991]]))])


In [64]:
y = model.forward(x)
#print(y)
print('For event %s' % in_name)
print('Tsunami potential output neurons: Yes=%s No=%s' % (float(y[0]), float(y[1])))
print('Tsunami potential: Yes=%s No=%s' % (float(y[0]) >= 0.5, float(y[1]) >= 0.5))


For event M 7.8 - Southwest of Sumatra, Indonesia - 2016-03-02T12:49:48.000Z
Tsunami potential output neurons: Yes=0.3773287236690521 No=0.7557135224342346
Tsunami potential: Yes=False No=True


In [66]:
torch.save(model, 'novianty2018.pt')

In [65]:
import torch.onnx
from torch.autograd import Variable

torch.onnx.export(model, Variable(x), 'novianty2018.onnx', verbose=True,
                  input_names = ['Bias, T0, Td, T0×Td, Mw'],
#                   input_names = [
#                      'Bias = 1',
#                      'Unnormalized rupture duration variable',
#                      'Unnormalized P-wave dominant period variable',
#                      'Unnormalized product of rupture duration and P-wave dominant period (T0×Td)',
#                      'Unnormalized moment magnitude (Mw)'],
                  output_names = ['y0=Tsunami-Yes, y1=Tsunami-No'])


graph(%Bias, T0, Td, T0×Td, Mw : Float(5)
      %1 : Float(2, 5)
      %2 : Float(2, 2)) {
  %3 : Float(5!, 2!) = onnx::Transpose[perm=[1, 0]](%1), scope: Sequential/Linear[0]
  %4 : Float(2) = onnx::MatMul(%Bias, T0, Td, T0×Td, Mw, %3), scope: Sequential/Linear[0]
  %5 : Float(2) = onnx::Sigmoid(%4), scope: Sequential/Sigmoid[1]
  %6 : Float(2!, 2!) = onnx::Transpose[perm=[1, 0]](%2), scope: Sequential/Linear[2]
  %7 : Float(2) = onnx::MatMul(%5, %6), scope: Sequential/Linear[2]
  %y0=Tsunami-Yes, y1=Tsunami-No : Float(2) = onnx::Sigmoid(%7), scope: Sequential/Sigmoid[3]
  return (%y0=Tsunami-Yes, y1=Tsunami-No);
}

